In [10]:
import pandas as pd
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from keras.wrappers.scikit_learn import KerasClassifier

from keras.models import Sequential
from keras.layers import Dense, Dropout

import numpy as np

from sklearn.preprocessing import StandardScaler

np.random.seed(7)

In [2]:
def prepare_data():
    pair = 'BTC_LTC'
    period = '5min'
    lag = 18
    
    df = pd.read_csv('../datasets/{}/{}.csv'.format(period, pair))
    close_lag_fields = ['close-{}'.format(n+1) for n in range(lag)]
    volume_lag_fields = ['volume-{}'.format(n+1) for n in range(lag)]
    df['datetime'] = pd.to_datetime(df['datetime'], unit='s')
    df['hour'] = df['datetime'].dt.hour
    df['dow'] = df['datetime'].dt.weekday
    df = pd.get_dummies(df, columns=['dow', 'hour'])
    dow_fields = [col for col in df if col.startswith('dow')]
    hour_fields = [col for col in df if col.startswith('hour')]
    for n in range(1, lag+1):
        df['close-' + str(n)] = df['close'] - df.shift(n)['close']
        df['volume-' + str(n)] = df['volume'] - df.shift(n)['volume']
    df['up'] = df['close'] < df.shift(-1)['close']
    df.dropna(inplace=True)
    X = df[close_lag_fields + volume_lag_fields + dow_fields + hour_fields].values
    y = df['up'].astype(int).values

    scaler = StandardScaler()

    X = scaler.fit_transform(X)
    
    y = np.asarray([np.eye(2)[v] for v in df['up'].astype(int).values])
    
    return X, y

In [3]:
X, y = prepare_data()

In [6]:
def create_model(dropout, loss):
    model = Sequential()
    model.add(Dense(125, input_dim=67, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(2, activation='softmax'))

    model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    return model

In [7]:
model = KerasClassifier(build_fn=create_model, verbose=1)

epochs = [20, 40, 80, 160]
batches = [8, 16, 32, 64]
dropouts = [0.0, 0.25, 0.75]
losses = ['binary_crossentropy', 'categorical_crossentropy']

param_grid = dict(dropout=dropouts, loss=losses, epochs=epochs, batch_size=batches)

grid = GridSearchCV(estimator=model, cv=2, param_grid=param_grid, verbose=2, n_jobs=-1)
grid_result = grid.fit(X, y)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

Fitting 2 folds for each of 96 candidates, totalling 192 fits
[CV] batch_size=8, dropout=0.0, loss=binary_crossentropy, epochs=20 ..
[CV] batch_size=8, dropout=0.0, loss=binary_crossentropy, epochs=20 ..
[CV] batch_size=8, dropout=0.0, loss=categorical_crossentropy, epochs=20 
[CV] batch_size=8, dropout=0.0, loss=categorical_crossentropy, epochs=20 
Epoch 1/20
Epoch 1/20
52551/52551 [==============================] - 16s - loss: 0.6974 - acc: 0.5078    
Epoch 2/20
50176/52551 [===========================>..] - ETA: 0s - loss: 0.6977 - acc: 0.5089
Epoch 2/20
52551/52551 [==============================] - 17s - loss: 0.6975 - acc: 0.5094    
Epoch 2/20

Epoch 2/20
52551/52551 [==============================] - 16s - loss: 0.6936 - acc: 0.5121    
Epoch 3/20
48488/52551 [==========================>...] - ETA: 1s - loss: 0.6937 - acc: 0.5139
Epoch 3/20
52551/52551 [==============================] - 17s - loss: 0.6372 - acc: 0.6437    
Epoch 3/20
52551/52551 [==============================]

52551/52551 [==============================] - 19s - loss: 0.6647 - acc: 0.5601    
Epoch 19/20
52551/52551 [==============================] - 18s - loss: 0.6654 - acc: 0.5544    
Epoch 19/20
52551/52551 [==============================] - 17s - loss: 0.6158 - acc: 0.6640    
Epoch 19/20
52551/52551 [==============================] - 21s - loss: 0.6626 - acc: 0.5602    
Epoch 20/20
52480/52551 [============================>.] - ETA: 0ss - loss: 0.7132 - acc: 0.49[CV]  batch_size=8, dropout=0.0, loss=categorical_crossentropy, epochs=20, total= 6.1min
[CV] batch_size=8, dropout=0.0, loss=binary_crossentropy, epochs=40 ..
41448/52551 [======================>.......] - ETA: 3s - loss: 0.6982 - acc: 0.5073[CV]  batch_size=8, dropout=0.0, loss=binary_crossentropy, epochs=20, total= 6.3min
[CV] batch_size=8, dropout=0.0, loss=categorical_crossentropy, epochs=40 
17688/52551 [=========>....................] - ETA: 10s - loss: 0.6591 - acc: 0.6099
Epoch 2/40
52551/52551 [========================

52551/52551 [==============================] - 18s - loss: 0.6811 - acc: 0.5419    
Epoch 13/40
12952/52551 [======>.......................] - ETA: 11s - loss: 0.6775 - acc: 0.5499
Epoch 13/40
52551/52551 [==============================] - 15s - loss: 0.6792 - acc: 0.5511    
Epoch 14/40
52551/52551 [==============================] - 14s - loss: 0.6782 - acc: 0.5473    
Epoch 14/40
52551/52551 [==============================] - 15s - loss: 0.6208 - acc: 0.6565    
Epoch 14/40
52551/52551 [==============================] - 14s - loss: 0.6198 - acc: 0.6585    
Epoch 15/40
52551/52551 [==============================] - 15s - loss: 0.6200 - acc: 0.6577    
Epoch 15/40
52551/52551 [==============================] - 15s - loss: 0.6741 - acc: 0.5557    
Epoch 16/40
52551/52551 [==============================] - 15s - loss: 0.6189 - acc: 0.6590    
Epoch 16/40
 2776/52551 [>.............................] - ETA: 13s - loss: 0.6693 - acc: 0.5692
Epoch 17/40
52551/52551 [=========================

27040/52551 [==============>...............] - ETA: 8s - loss: 0.6349 - acc: 0.5875
Epoch 29/40
52551/52551 [==============================] - 17s - loss: 0.6374 - acc: 0.5841    
Epoch 30/40
52551/52551 [==============================] - 15s - loss: 0.6225 - acc: 0.5983    
Epoch 31/40
52551/52551 [==============================] - 15s - loss: 0.6014 - acc: 0.6699    
Epoch 31/40
52551/52551 [==============================] - 16s - loss: 0.6032 - acc: 0.6713    
Epoch 30/40
52551/52551 [==============================] - 16s - loss: 0.6219 - acc: 0.5994    
Epoch 32/40
52551/52551 [==============================] - 16s - loss: 0.5996 - acc: 0.6705    
Epoch 32/40
52551/52551 [==============================] - 17s - loss: 0.6030 - acc: 0.6698    
Epoch 31/40
48040/52551 [==========================>...] - ETA: 1s - loss: 0.5989 - acc: 0.6730
Epoch 33/40
52551/52551 [==============================] - 16s - loss: 0.6021 - acc: 0.6713    
Epoch 32/40
52551/52551 [===========================

52551/52551 [==============================] - 19s - loss: 0.6498 - acc: 0.6301    
Epoch 2/80
 3880/52551 [=>............................] - ETA: 18s - loss: 0.6385 - acc: 0.6495
Epoch 4/80
52551/52551 [==============================] - 19s - loss: 0.6353 - acc: 0.6476    
Epoch 4/80
52551/52551 [==============================] - 18s - loss: 0.6933 - acc: 0.5144    
Epoch 3/80
44968/52551 [========================>.....] - ETA: 2s - loss: 0.6925 - acc: 0.5132
Epoch 3/80
52551/52551 [==============================] - 18s - loss: 0.6332 - acc: 0.6496    
Epoch 5/80
52551/52551 [==============================] - 20s - loss: 0.6926 - acc: 0.5130    
Epoch 5/80
52551/52551 [==============================] - 18s - loss: 0.6351 - acc: 0.6470    
Epoch 4/80
52551/52551 [==============================] - 19s - loss: 0.6315 - acc: 0.6496    
Epoch 6/80
52551/52551 [==============================] - 18s - loss: 0.6920 - acc: 0.5196    
Epoch 5/80
43368/52551 [=======================>......] - ET

...

52551/52551 [==============================] - 3s - loss: 0.6933 - acc: 0.5101     
Epoch 149/160
52551/52551 [==============================] - 3s - loss: 0.6371 - acc: 0.6461     
Epoch 147/160
37696/52551 [====================>.........] - ETA: 0s - loss: 0.6932 - acc: 0.5093
Epoch 145/160
52551/52551 [==============================] - 3s - loss: 0.6932 - acc: 0.5104     
Epoch 146/160
14720/52551 [=======>......................] - ETA: 2s - loss: 0.6388 - acc: 0.6393
Epoch 146/160
52551/52551 [==============================] - 3s - loss: 0.6936 - acc: 0.5109     
Epoch 147/160
52551/52551 [==============================] - 3s - loss: 0.6357 - acc: 0.6436     
Epoch 149/160
52551/52551 [==============================] - 3s - loss: 0.6931 - acc: 0.5104     
Epoch 152/160
52551/52551 [==============================] - 3s - loss: 0.6362 - acc: 0.6453     
Epoch 150/160
18752/52551 [=========>....................] - ETA: 2s - loss: 0.6354 - acc: 0.6495
Epoch 148/160
52551/52551 [=======

49984/52551 [===========================>..] - ETA: 0s[CV]  batch_size=64, dropout=0.75, loss=binary_crossentropy, epochs=160, total= 9.5min


[Parallel(n_jobs=-1)]: Done 192 out of 192 | elapsed: 591.7min finished


Epoch 1/40
105102/105102 [==============================] - 4s - loss: 0.7210 - acc: 0.5250     
Epoch 2/40
105102/105102 [==============================] - 4s - loss: 0.6916 - acc: 0.5355     
Epoch 3/40
105102/105102 [==============================] - 4s - loss: 0.6915 - acc: 0.5364     
Epoch 4/40
105102/105102 [==============================] - 4s - loss: 0.6913 - acc: 0.5357     
Epoch 5/40
105102/105102 [==============================] - 4s - loss: 0.6911 - acc: 0.5361     
Epoch 6/40
105102/105102 [==============================] - 4s - loss: 0.6912 - acc: 0.5360     
Epoch 7/40
105102/105102 [==============================] - 4s - loss: 0.6915 - acc: 0.5365     
Epoch 8/40
105102/105102 [==============================] - 4s - loss: 0.6913 - acc: 0.5360     
Epoch 9/40
105102/105102 [==============================] - 4s - loss: 0.6912 - acc: 0.5358     
Epoch 10/40
105102/105102 [==============================] - 4s - loss: 0.6912 - acc: 0.5362     
Epoch 11/40
105102/105102 [==

/home/vampire/PycharmProjects/mikasa_ed/venv/lib/python3.5/site-packages/sklearn/model_selection/_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [38]:
def validate_model(model, X_train, y_train, X_test, y_test, epochs, batch_size):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.33, verbose=0)
    scores = model.evaluate(X_test, y_test, verbose=0)
    score = scores[1]
    return score

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [33]:
model = create_model(0.75, 'binary_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=40, batch_size=32)
print()
print('Score: ', score)

19232/21021 [==========================>...] - ETA: 0s
Score:  0.537652823339


In [34]:
model = create_model(0.75, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=40, batch_size=32)
print()
print('Score: ', score)

20576/21021 [============================>.] - ETA: 0s
Score:  0.537652823339


In [35]:
model = create_model(0.75, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=40, batch_size=64)
print()
print('Score: ', score)

20160/21021 [===========================>..] - ETA: 0s
Score:  0.565624851336


In [36]:
model = create_model(0.75, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=80, batch_size=64)
print()
print('Score: ', score)

21021/21021 [==============================] - 0s     

Score:  0.574901289159


In [37]:
model = create_model(0.75, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=80, batch_size=32)
print()
print('Score: ', score)

20224/21021 [===========================>..] - ETA: 0s
Score:  0.537652823339


In [39]:
model = create_model(0.75, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=160, batch_size=64)
print()
print('Score: ', score)


Score:  0.577708006302


In [43]:
model = create_model(0.5, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=160, batch_size=64)
model.save('keras_5min_lag_18_BTC_LTC.h5')
print('Score: ', score)

Score:  0.581275867013


In [44]:
model = create_model(0.5, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=320, batch_size=64)
print('Score: ', score)

Score:  0.567480138896


In [42]:
model = create_model(0.25, 'categorical_crossentropy')
score = validate_model(model, X_train, y_train, X_test, y_test, epochs=200, batch_size=64)
print('Score: ', score)

Score:  0.576090576078
